### Calculating distributions of different properties from Ariane output
This notebook has examples for calculating 1, 2, and 3D distributions from Ariane trajectory output.  
The usage builds on the functions in ariane_distributionfunctions.  
The distributions are saved in `data/processed/lagrangian/`

#### Load the data and define some variables

In [1]:
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
import xarray as xr
from xhistogram.xarray import histogram as hist
from thermodynamics import calc_sigmantr
from ariane_distributionfunctions import ariane_D1, ariane_D2, ariane_D3, ariane_D4

In [2]:
rootdir = '/home/ocean_personal_data/graemem/ariane/'
model = 'orca025_global_5d'
experiment = 'quant_back_seedNAn1_t*_sign27.7-28_MLrefz8delsig0.01'
filepath = rootdir+'experiments/'+model+'/'+experiment+'/ariane_positions_quantitative.nc'
filepath_initial = rootdir+'experiments/'+model+'/'+experiment+'/ariane_initial.nc'
filepath_time = rootdir+'time/time_orca025_global_5d.mat'
filepath_region = rootdir+'experiments/'+model+'/quant_back_seedNAn1_t3560-sep-4217_sign27.7-28_MLrefz8delsig0.01/region_limits'
# Universal variables
spy = 365*24*60*60
yrst = 1958
yrend = 2016
ventsec = 7
lastinit = 4217

In [5]:
# Ariane input
ds_initial = xr.open_mfdataset(filepath_initial,combine='nested',concat_dim='ntraj')
ds_initial.init_volume.name = 'init_volume'
# Ariane output
ds = xr.open_mfdataset(filepath,combine='nested',concat_dim='ntraj')
ds = xr.merge([ds, ds_initial.init_volume])
ds['final_age'] = ds.final_age.astype('timedelta64[s]').astype('float64')/spy
ds['init_dens'] = calc_sigmantr(ds.init_temp,ds.init_salt)
ds['final_dens'] = calc_sigmantr(ds.final_temp,ds.final_salt)
# Model times
time_vals = np.append(np.array([0]),sio.loadmat(filepath_time)['time'].squeeze())
time = xr.DataArray(time_vals,dims=['nfile'],coords={'nfile':np.arange(time_vals.size)})
# Reagion limits
region_limits = np.loadtxt(filepath_region)

In [7]:
# Bins
years = np.arange(yrst,yrend+1)
ages = np.arange(-3/12,yrend-yrst+9/12)
densities = np.arange(27.7,28,0.01)
init_t_unique = np.unique(ds.init_t)
inits = np.append(init_t_unique-0.5,init_t_unique[-1]+0.5)
xs = np.arange(region_limits[0,0],region_limits[0,1])
ys = np.arange(region_limits[1,0],region_limits[1,1])
lats = np.arange(-30,80,5)

#### A 1-D distribution

In [15]:
var1 = ds.final_age
bins = [ages]
weights = ds.init_volume
S = (ds.final_section==ventsec) & (ds.init_t==lastinit)
subsetname = 'final_section-7_init_t-last'

D1 = ariane_D1(
    var1=var1,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    save=True
        )

#### A 2-D distribution

In [9]:
var1 = ds.final_age
var2 = ds.init_dens
bins = [ages,densities]
weights = ds.init_volume
#weights.name='iViL'
S = (ds.final_section==ventsec)
subsetname = 'final_section-7'

D2 = ariane_D2(
    var1=var1,
    var2=var2,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    save=True
        )

#### A 3-D distribution

In [18]:
var1 = ds.init_lat
var2 = ds.final_age
var3 = ds.init_dens
bins = [,ys,ages]
weights = ds.init_volume
# weights.name='iViT'
S = (ds.final_section==ventsec) & (ds.init_t==lastinit)
subsetname = 'final_section-7_init_t-last'

D3 = ariane_D3(
    var1=var1,
    var2=var2,
    var3=var3,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    verbose=True,
    save=True
        )

final_age : -0.25 to 0.75
final_age : 0.75 to 1.75
final_age : 1.75 to 2.75
final_age : 2.75 to 3.75
final_age : 3.75 to 4.75
final_age : 4.75 to 5.75
final_age : 5.75 to 6.75
final_age : 6.75 to 7.75
final_age : 7.75 to 8.75
final_age : 8.75 to 9.75
final_age : 9.75 to 10.75
final_age : 10.75 to 11.75
final_age : 11.75 to 12.75
final_age : 12.75 to 13.75
final_age : 13.75 to 14.75
final_age : 14.75 to 15.75
final_age : 15.75 to 16.75
final_age : 16.75 to 17.75
final_age : 17.75 to 18.75
final_age : 18.75 to 19.75
final_age : 19.75 to 20.75
final_age : 20.75 to 21.75
final_age : 21.75 to 22.75
final_age : 22.75 to 23.75
final_age : 23.75 to 24.75
final_age : 24.75 to 25.75
final_age : 25.75 to 26.75
final_age : 26.75 to 27.75
final_age : 27.75 to 28.75
final_age : 28.75 to 29.75
final_age : 29.75 to 30.75
final_age : 30.75 to 31.75
final_age : 31.75 to 32.75
final_age : 32.75 to 33.75
final_age : 33.75 to 34.75
final_age : 34.75 to 35.75
final_age : 35.75 to 36.75
final_age : 36.75 to 

### A 4-D distribution
This could take a considerable amount of time to run. See calc_D4.py for a python script that can be run outside of the notebook environment.

In [ ]:
var1 = ds.final_x
var2 = ds.final_y
var3 = ds.final_age
var4 = ds.init_t

bins = [xs,ys,ages,inits]
weights = ds.init_volume

S = (ds.final_section==ventsec)
subsetname = 'final_section-7'

D4 = ariane_D4(
    var1=var1,
    var2=var2,
    var3=var3,
    var4=var4,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    verbose=True,
    save=True
        )

init_t : 1367.5 to 1440.5
end
end
init_t : 1440.5 to 1513.5
